# Using the lcvr_learning Class to Measure Training Data

To train our model, it helps to have a large amount of training data. While this can be measured and calibrated by hand, it can take several hours for a single wavelength to get the data. On top of this, changing the orientation of the LCVR's will then change the necessary driving voltage for different polarizations, resulting in prior training data being useless. To combat this, the class has an automated function for collecting training data.

First we need to import the library. In addition, we set the device to our input channel on our ADC module. This is required to know where the data is being read from. We can also set the function generator being used for the square wave as well, which *should* work so long as it follows the same SCPI command syntax as the Siglent 2042x. Otherwise the functions that read and write parameters to the function generators may need to be modified.

In [1]:
import lcvr_learning as lcl
import numpy as np
import pyvisa
import pandas as pd
rm = pyvisa.ResourceManager('@py')
lcvrs = lcl.lcvr_learning(0x6a,0x6b)

There's a few functions in the class that simplify some processes, such as setting the input voltage, getting information on the current inputs, and reading out data from the ADC. This just cuts back a bit on SCPI queries and hopefully makes the code more readable. As an example:

In [14]:
lcvrs.get_voltage()

-0.33962500000000007

Returns the current output voltage on the ADC, and

In [15]:
lcvrs.get_wave_info(1)
lcvrs.get_wave_info(2)

(2000.0, 1.0)

Returns the frequency and amplitude of the inputs on channel 1 and 2 respectively. These are initialized to 2000 Hz and 1 V when the connection is made as a safeguard. 

# Automatically Reading and Writing Data:

The class has the function get_training_data(iterations,wavelength), which will automatically alter the input voltages on each LCVR and read the differential output on two photodetectors that are preceeded by a Wollaston prism. This in turn means the output voltage is directly correlated to the polarization state of the output light. All of this data is then written to a pandas dataframe for easy manipulation.

In [11]:
data = lcvrs.get_training_data(200,480)

Starting training data scan. Don't touch anything please


In [12]:
data.head()

,Wavelength,V1,V2,Out
0,480,1.000000,1.0,-0.335938
1,480,1.061224,1.0,-0.336234
2,480,1.122449,1.0,-0.336219
3,480,1.183673,1.0,-0.336219
4,480,1.244898,1.0,-0.336250


Here we write the data to a .csv file to train our model on later. The name is simply the wavelength of light used (in nm) to try and keep things readable.

In [13]:
data.to_csv('trainingdata_new/480.csv')